In [15]:
import nltk
from nltk.corpus import wordnet
#nltk.download('wordnet')
#nltk.download('omw-1.4')
import os
os.environ['CURL_CA_BUNDLE'] = ''

In [16]:
import os
os.environ['CURL_CA_BUNDLE'] = ''
import numpy as np
import os
import json
import torch
import random
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split

# Configurações iniciais
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
file_path = '/Users/Administrador/Documents/GitHub/CardioBERTpt/labels_anotados.json'
max_len = 512
batch_size = 4

# Inicializando o tokenizer e o modelo
tokenizer = AutoTokenizer.from_pretrained("pucpr-br/cardiobertpt")
model = AutoModelForTokenClassification.from_pretrained("pucpr-br/cardiobertpt")


def synonym_replacement(text, labels):
    nltk.download('averaged_perceptron_tagger')
    words = nltk.word_tokenize(text)
    new_words = words.copy()
    pos_tags = nltk.pos_tag(words)
    
    for i, word in enumerate(words):
        if labels[i] == 'O':  # Only replace non-entity words
            # Check part-of-speech tag and only replace nouns, adjectives, verbs, and adverbs
            if pos_tags[i][1] in {'NN', 'JJ', 'VB', 'RB'}:
                synonyms = get_synonyms(word)
                if synonyms:
                    synonym = random.choice(synonyms)
                    new_words[i] = synonym
    return ' '.join(new_words), labels

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonym = lemma.name().replace('_', ' ').replace('-', ' ').lower()
            synonyms.add(synonym)
    if word in synonyms:
        synonyms.remove(word)  # Remove the original word
    return list(synonyms)


# Função para pré-processamento de dados
def preprocess_data(data, tokenizer, augmentation=False):
    texts, labels = [], []
    for entry in data:
        text, label = process_entry(entry, tokenizer)
        if augmentation:
            text, label = synonym_replacement(text, label)
        texts.append(text)
        labels.append(label)
    return texts, labels

def process_entry(entry, tokenizer):
    text = entry['data']['text']
    annotations = entry['annotations'][0]['result']
    text_labels = ['O'] * len(text)
    for annotation in annotations:
        update_labels(annotation, text_labels)
    aligned_labels = align_labels(text, text_labels, tokenizer)
    return text, aligned_labels

def update_labels(annotation, text_labels):
    start, end = annotation['value']['start'], annotation['value']['end']
    label = annotation['value']['labels'][0]
    for i in range(start, end):
        prefix = 'B-' if i == start else 'I-'
        text_labels[i] = prefix + label

def align_labels(text, text_labels, tokenizer):
    tokenized_input = tokenizer(text, return_offsets_mapping=True)
    word_ids = tokenized_input.word_ids()
    return [-100 if i is None else text_labels[i] for i in word_ids]

# Definição da classe CustomDataset
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len, label_map):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.label_map = label_map

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        labels = self.labels[idx]

        # Convertendo rótulos textuais para inteiros
        converted_labels = [self.label_map.get(label, -100) for label in labels]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids = encoding['input_ids'].flatten()
        attention_mask = encoding['attention_mask'].flatten()

        # Ajustar o comprimento dos rótulos e criar tensor
        converted_labels += [-100] * (self.max_len - len(converted_labels))
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(converted_labels[:self.max_len], dtype=torch.long)
        }


# Carregando e pré-processando os dados com augmentation ativada
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)
texts, labels = preprocess_data(data, tokenizer, augmentation=False)

# Dividindo dados em treino e validação
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.1)

# Mapeamento de rótulos
label_map = {'O': 0, 'B-C': 1, 'B-DT': 2, 'B-N': 3, 'I-C': 4, 'I-DT': 5, 'I-N': 6}
train_dataset = CustomDataset(train_texts, train_labels, tokenizer, max_len, label_map)
validation_dataset = CustomDataset(val_texts, val_labels, tokenizer, max_len, label_map)


model = AutoModelForTokenClassification.from_pretrained("pucpr-br/cardiobertpt", num_labels=len(label_map))

label_map_reverse = {v: k for k, v in label_map.items()}

from seqeval.metrics import precision_score, recall_score, f1_score

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Converte os índices de previsões para rótulos de texto
    true_predictions = [
        [label_map_reverse[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_map_reverse[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    # Calcula as métricas usando seqeval
    return {
        "precision": precision_score(true_labels, true_predictions),
        "recall": recall_score(true_labels, true_predictions),
        "f1": f1_score(true_labels, true_predictions),
    }

# Restante do código...

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=30,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",  # Adicione esta linha para avaliar no final de cada época
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics  # Você precisará definir uma função de métricas
)
trainer.train()


# Caminho para salvar o modelo e o tokenizer
model_save_path = "/Users/Administrador/Documents/GitHub/CardioBERTpt/model"

# Salvar modelo e tokenizer
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)
import pandas as pd

# Substitua com o caminho do seu arquivo, se necessário
df = pd.read_excel("/Users/Administrador/Documents/GitHub/ENTClinPT/df_tropo_avc_shig_27.12.2023.xlsx")

# Vamos assumir que a coluna com os textos é 'DS_RESPOSTA'
textos = df['DS_RESPOSTA'].tolist()
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained(model_save_path)
max_len = 512  # Definido durante o treinamento

encoded_texts = [tokenizer.encode(text, max_length=max_len, truncation=True, return_tensors="pt") for text in textos]
# Carregue o modelo treinado (substitua pelo caminho do seu modelo se estiver salvando localmente)
model = AutoModelForTokenClassification.from_pretrained(model_save_path)

model.eval()  # Colocar o modelo em modo de avaliação

predictions = []
for encoded_text in encoded_texts:
    with torch.no_grad():
        output = model(encoded_text)
        predictions.append(output.logits.argmax(dim=-1))
import pandas as pd

# Converter os IDs dos rótulos de volta para rótulos textuais
label_map_reverse = {v: k for k, v in label_map.items()}

# Certifique-se de que df é o seu DataFrame
# Inicialize a coluna Extracted_Entities se ela não existir
if 'Extracted_Entities' not in df.columns:
    df['Extracted_Entities'] = pd.NA

for index, row in df.iterrows():
    # Extrair o texto da coluna DS_RESPOSTA
    text = row['DS_RESPOSTA']

    # Processar o texto
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_len, return_tensors="pt")

    # Fazer previsões
    outputs = model(**inputs)
    predictions = outputs.logits.argmax(dim=-1)

    # Extrair entidades
    def extract_entities(predictions, input_ids, tokenizer):
        tokens = tokenizer.convert_ids_to_tokens(input_ids.squeeze().tolist())
        labels = predictions.squeeze().tolist()
        entities = []
        for idx, label_id in enumerate(labels):
            label = label_map_reverse.get(label_id)
            if label and label != "O":
                entities.append((tokens[idx], label))
        return entities

    # Aplicar a função de extração de entidades
    extracted_entities = extract_entities(predictions, inputs['input_ids'], tokenizer)

    # Adicionar as entidades extraídas ao DataFrame
    df.at[index, 'Extracted_Entities'] = extracted_entities

# Exibir o DataFrame
print(df)
df.to_csv('tokens.csv')

from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from seqeval.metrics import classification_report
from seqeval.scheme import IOB2
import numpy as np

# Isso deve ser uma lista ou um dicionário com chaves numéricas.
label_list = ['O', 'B-C', 'B-DT', 'B-N', 'I-C', 'I-DT', 'I-N']

# Inicialize o Trainer
trainer = Trainer(model=model)

# Fazer previsões no conjunto de validação
predictions, labels, _ = trainer.predict(validation_dataset)

# As previsões estão no formato logits, você precisa aplicar argmax para obter os IDs das previsões
predictions = np.argmax(predictions, axis=2)

# Converter os IDs das previsões e rótulos para os rótulos textuais
def align_predictions(predictions, label_ids):
    preds = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, label_ids)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, label_ids)]
    return preds, true_labels

# Obter previsões e rótulos verdadeiros alinhados
preds, true_labels = align_predictions(predictions, labels)

# Use seqeval para calcular as métricas
report = classification_report(true_labels, preds, mode='strict', scheme=IOB2)

# Gerar matriz de confusão
flat_preds = [p for sublist in preds for p in sublist]
flat_true_labels = [l for sublist in true_labels for l in sublist]

# Gerar matriz de confusão
conf_matrix = confusion_matrix(flat_true_labels, flat_preds, labels=label_list)

# Calcular especificidade
specificity_list = []
for i, label in enumerate(label_list):
    tn = conf_matrix[i,i]
    fp = conf_matrix[:,i].sum() - tn
    fn = conf_matrix[i,:].sum() - tn
    specificity = tn / (tn + fp) if tn + fp else 0
    specificity_list.append(specificity)

# Calcula precisão, recall e f1-score
precision, recall, f1, _ = precision_recall_fscore_support(flat_true_labels, flat_preds, labels=label_list)

# Adicione especificidade ao relatório
print(report)
print("\nEspecificidade por classe:")
for i, label in enumerate(label_list):
    print(f"{label}: {specificity_list[i]:.2f}")


Some weights of BertForTokenClassification were not initialized from the model checkpoint at pucpr-br/cardiobertpt and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Token indices sequence length is longer than the specified maximum sequence length for this model (918 > 512). Running this sequence through the model will result in indexing errors
Some weights of BertForTokenClassification were not initialized from the model checkpoint at pucpr-br/cardiobertpt and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  1%|          | 11/1740 [00:02<05:16,  5.45it/s]

{'loss': 1.9276, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.17}


  1%|          | 21/1740 [00:04<05:11,  5.52it/s]

{'loss': 1.8403, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.34}


  2%|▏         | 31/1740 [00:05<05:10,  5.50it/s]

...

{'loss': 0.0486, 'learning_rate': 0.0, 'epoch': 30.0}


                                                   
100%|██████████| 1740/1740 [07:31<00:00,  3.86it/s]


{'eval_loss': 1.0522546768188477, 'eval_precision': 0.3373493975903614, 'eval_recall': 0.6363636363636364, 'eval_f1': 0.4409448818897638, 'eval_runtime': 0.5064, 'eval_samples_per_second': 51.346, 'eval_steps_per_second': 13.824, 'epoch': 30.0}
{'train_runtime': 451.1051, 'train_samples_per_second': 15.362, 'train_steps_per_second': 3.857, 'train_loss': 0.32177342853144925, 'epoch': 30.0}
      Coluna1  CD_PACIENTE  CD_ATENDIMENTO_novo        DH_DOCUMENTO  \
0           0      1092064               795807 2023-01-11 12:20:05   
1           3      1091806               790465 2023-01-11 12:28:00   
2           4       785747               795897 2023-01-13 00:06:38   
3           7      1063053               801158 2023-01-14 10:51:29   
4          15      1092391               801246 2023-01-15 18:22:00   
...       ...          ...                  ...                 ...   
1559    14683       731440              1237062 2023-12-26 13:10:00   
1560    14689       823876              

100%|██████████| 4/4 [00:00<00:00, 14.16it/s]

              precision    recall  f1-score   support

           C       0.46      0.68      0.55        25
          DT       0.37      0.47      0.41        15
           N       0.80      1.00      0.89         4

   micro avg       0.46      0.64      0.53        44
   macro avg       0.54      0.72      0.62        44
weighted avg       0.46      0.64      0.53        44


Especificidade por classe:
O: 0.98
B-C: 0.46
B-DT: 0.37
B-N: 0.80
I-C: 0.47
I-DT: 0.44
I-N: 0.75


In [17]:
print(report)

              precision    recall  f1-score   support

           C       0.46      0.68      0.55        25
          DT       0.37      0.47      0.41        15
           N       0.80      1.00      0.89         4

   micro avg       0.46      0.64      0.53        44
   macro avg       0.54      0.72      0.62        44
weighted avg       0.46      0.64      0.53        44



In [18]:
df.to_csv('tokens40.csv', index=False)